In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('heart.csv')
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [3]:
categorical_val = []
continuous_val = []
for col in df.columns:
    if len(df[col].unique()) <= 10:
        categorical_val.append(col)
    else:
        continuous_val.append(col)

In [4]:
categorical_val.remove('target')
dataset = pd.get_dummies(df, columns = categorical_val)
dataset.head()

,age,trestbps,chol,thalach,oldpeak,target,sex_0,sex_1,cp_0,cp_1,...,slope_2,ca_0,ca_1,ca_2,ca_3,ca_4,thal_0,thal_1,thal_2,thal_3
0,63,145,233,150,2.3,1,0,1,0,0,...,0,1,0,0,0,0,0,1,0,0
1,37,130,250,187,3.5,1,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
2,41,130,204,172,1.4,1,1,0,0,1,...,1,1,0,0,0,0,0,0,1,0
3,56,120,236,178,0.8,1,0,1,0,1,...,1,1,0,0,0,0,0,0,1,0
4,57,120,354,163,0.6,1,1,0,1,0,...,1,1,0,0,0,0,0,0,1,0


In [5]:
from sklearn.preprocessing import StandardScaler

s_sc = StandardScaler()
col_to_scale = ['age', 'trestbps', 'chol', 'thalach', 'oldpeak']
dataset[col_to_scale] = s_sc.fit_transform(dataset[col_to_scale])
dataset.head()

,age,trestbps,chol,thalach,oldpeak,target,sex_0,sex_1,cp_0,cp_1,...,slope_2,ca_0,ca_1,ca_2,ca_3,ca_4,thal_0,thal_1,thal_2,thal_3
0,0.952197,0.763956,-0.256334,0.015443,1.087338,1,0,1,0,0,...,0,1,0,0,0,0,0,1,0,0
1,-1.915313,-0.092738,0.072199,1.633471,2.122573,1,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
2,-1.474158,-0.092738,-0.816773,0.977514,0.310912,1,1,0,0,1,...,1,1,0,0,0,0,0,0,1,0
3,0.180175,-0.663867,-0.198357,1.239897,-0.206705,1,0,1,0,1,...,1,1,0,0,0,0,0,0,1,0
4,0.290464,-0.663867,2.082050,0.583939,-0.379244,1,1,0,1,0,...,1,1,0,0,0,0,0,0,1,0


In [6]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def print_score(clf, x_train, y_train, x_test, y_test, train):
    if train == True:
        pred = clf.predict(x_train)
        clf_report = pd.DataFrame(classification_report(y_train, pred, output_dict=True))
        print(f'Accuracy Score: {accuracy_score(y_train, pred) * 100:.4f}%')
        print('______________________________________________________________________')
        print(f'Classification Report:\n{clf_report}')
#         print('______________________________________________________________________')
#         print(f'Confusion Matrix: \n {confusion_matrix(y_train, pred)}\n')
    elif train == False:
        pred = clf.predict(x_test)
        clf_report = pd.DataFrame(classification_report(y_test, pred, output_dict=True))
        print(f'Accuracy Score: {accuracy_score(y_test, pred) * 100:.4f}%')
        print('______________________________________________________________________')
        print(f'Classification Report:\n{clf_report}')
#         print('______________________________________________________________________')
#         print(f'Confusion Matrix: \n {confusion_matrix(y_test, pred)}\n')

In [7]:
from sklearn.model_selection import train_test_split

x = dataset.drop('target', axis=1)
y = dataset.target
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=42)

In [8]:
from sklearn.tree import DecisionTreeClassifier

dt_clf = DecisionTreeClassifier(random_state=42).fit(x_train, y_train)

In [9]:
print_score(dt_clf, x_train, y_train, x_test, y_test, train=True)

Accuracy Score: 100.0000%
______________________________________________________________________
Classification Report:
              0      1  accuracy  macro avg  weighted avg
precision   1.0    1.0       1.0        1.0           1.0
recall      1.0    1.0       1.0        1.0           1.0
f1-score    1.0    1.0       1.0        1.0           1.0
support    97.0  115.0       1.0      212.0         212.0


In [10]:
print_score(dt_clf, x_train, y_train, x_test, y_test, train=False)

Accuracy Score: 78.0220%
______________________________________________________________________
Classification Report:
                   0          1  accuracy  macro avg  weighted avg
precision   0.723404   0.840909   0.78022   0.782157      0.787967
recall      0.829268   0.740000   0.78022   0.784634      0.780220
f1-score    0.772727   0.787234   0.78022   0.779981      0.780698
support    41.000000  50.000000   0.78022  91.000000     91.000000


In [11]:
from sklearn.model_selection import GridSearchCV

params = {'criterion': ('gini', 'entropy'), 
          'splitter': ('best', 'random'), 
          'max_depth': (list(range(1, 20))), 
          'min_samples_split': [2, 3, 4], 
          'min_samples_leaf': list(range(1, 20))
          }
# dt_clf = DecisionTreeClassifier(random_state=42)
dt_cv = GridSearchCV(dt_clf, params, scoring='accuracy', n_jobs=-1, verbose=1, cv=3).fit(x_train, y_train)
best_params = dt_cv.best_params_
print(f'Best_params: {best_params}')

Fitting 3 folds for each of 4332 candidates, totalling 12996 fits
Best_params: {'criterion': 'entropy', 'max_depth': 5, 'min_samples_leaf': 2, 'min_samples_split': 2, 'splitter': 'random'}


In [12]:
dt_clf_tuning = DecisionTreeClassifier(**best_params).fit(x_train, y_train)

In [13]:
print_score(dt_clf_tuning, x_train, y_train, x_test, y_test, train=True)

Accuracy Score: 91.9811%
______________________________________________________________________
Classification Report:
                   0           1  accuracy   macro avg  weighted avg
precision   0.900000    0.937500  0.919811    0.918750      0.920342
recall      0.927835    0.913043  0.919811    0.920439      0.919811
f1-score    0.913706    0.925110  0.919811    0.919408      0.919892
support    97.000000  115.000000  0.919811  212.000000    212.000000


In [14]:
print_score(dt_clf_tuning, x_train, y_train, x_test, y_test, train=False)

Accuracy Score: 76.9231%
______________________________________________________________________
Classification Report:
                   0          1  accuracy  macro avg  weighted avg
precision   0.692308   0.871795  0.769231   0.782051      0.790927
recall      0.878049   0.680000  0.769231   0.779024      0.769231
f1-score    0.774194   0.764045  0.769231   0.769119      0.768617
support    41.000000  50.000000  0.769231  91.000000     91.000000


In [15]:
dt_clf_train = accuracy_score(y_train, dt_clf.predict(x_train)) * 100
dt_clf_test = accuracy_score(y_test, dt_clf.predict(x_test)) * 100
dt_clf_tuning_train = accuracy_score(y_train, dt_clf_tuning.predict(x_train)) * 100
dt_clf_tuning_test = accuracy_score(y_test, dt_clf_tuning.predict(x_test)) * 100

result = pd.DataFrame(columns=['Model', 'Non-tuning train accuracy %', 'Tuning train accuracy %', 'Non-tuning test accuracy %', 'Tuning test accuracy %'])
dt_result = pd.DataFrame(data=[['Logistic regression', dt_clf_train, dt_clf_tuning_train, dt_clf_test, dt_clf_tuning_test]],
                        columns=['Model', 'Non-tuning train accuracy %', 'Tuning train accuracy %', 'Non-tuning test accuracy %', 'Tuning test accuracy %'])
result = result.append(dt_result, ignore_index=True)
result

,Model,Non-tuning train accuracy %,Tuning train accuracy %,Non-tuning test accuracy %,Tuning test accuracy %
0,Logistic regression,100.0,91.981132,78.021978,76.923077
